The first attempt is to make the computer generates all outputs randomly and analysing the win rate of the computer. This is the control group. The objective is to get win rates higher than if all is choosen at random.  

#### Imports

In [85]:
from enum import Enum
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#### Preparation

In [86]:
df = pd.read_csv('data.csv')

class Result(Enum):
    Win = [1, 0, 0]
    Lost = [0, 1, 0]
    Tie = [0, 0, 1]

class Pick(Enum): 
    Paper = [1, 0, 0]
    Rock = [0, 1, 0]
    Scissors = [0, 0, 1]

#### Evaluate WinRate of raw data

In [87]:
def play(human, computer, verbose = False): 
    total, won, lost = 0, 0, 0
    results = []
    for i in range (0, len(df)):
        match human[i]: 
            case 'R': 
                match computer[i]: 
                    case 'P': 
                        total += 1
                        won += 1
                        results.append(Result.Win.value)
                    case 'S': 
                        total += 1
                        lost += 1
                        results.append(Result.Lost.value)
                    case 'R':
                        results.append(Result.Tie.value)
            case 'P':
                match computer[i]: 
                    case 'S': 
                        total += 1
                        won += 1
                        results.append(Result.Win.value)
                    case 'R': 
                        total += 1
                        lost += 1
                        results.append(Result.Lost.value)
                    case 'P':
                        results.append(Result.Tie.value)
            case 'S': 
                match computer[i]: 
                    case 'P': 
                        total += 1
                        lost += 1
                        results.append(Result.Lost.value)
                    case 'R': 
                        total += 1
                        won += 1
                        results.append(Result.Win.value)
                    case 'S':
                        results.append(Result.Tie.value)
    if (verbose): 
        # evaluation: 
        print(f"Total games: {total}")
        print(f"Won: {won}")
        print(f"Lost: {lost}")
        print(f"Win rate: {won/total*100}%")

    # winrate and the results array
    return won/total, results



In [88]:
for i in range(1, 7):
    print(play(df[f'H{i}'], df[f'C{i}'])[0])


0.65625
0.375
0.3076923076923077
0.35714285714285715
0.4583333333333333
0.52


#### Create Tensor for One round

In [89]:
print(Pick.Rock.value + Pick.Paper.value + [1])

[0, 1, 0, 1, 0, 0, 1]


In [90]:
def play(human, computer, verbose = False): 
    for i in range (0, len(df)):
        match human[i]: 
            case 'R': 
                match computer[i]: 
                    case 'P': 
                        return Pick.Rock.value + Pick.Paper.value + [1] # win
                    case 'S': 
                        return Pick.Rock.value + Pick.Scissors.value + [0] # loss
                    case 'R':
                        return Pick.Rock.value + Pick.Rock.value + [0] # tie
            case 'P':
                match computer[i]: 
                    case 'S': 
                        return Pick.Paper.value + Pick.Scissors.value + [1] # win
                    case 'R': 
                        return Pick.Paper.value + Pick.Rock.value + [0] # loss
                    case 'P':
                        return Pick.Paper.value + Pick.Paper.value + [0] # tie
            case 'S': 
                match computer[i]: 
                    case 'P': 
                        return Pick.Scissors.value + Pick.Paper.value + [0] # loss
                    case 'R': 
                        return Pick.Scissors.value + Pick.Rock.value + [1] # win
                    case 'S':
                        return Pick.Scissors.value + Pick.Scissors.value + [0] # tie

In [91]:
data_1 = [play(df[f'H1'][i], df[f'C1'][i]) for i in range(len(df))]
data_1 = pd.DataFrame(data_1, columns=['HP', 'HR', 'HS', 'CP', 'CR', 'CS', 'Win'])

print(data_1.head())

   HP  HR  HS  CP  CR  CS  Win
0   0   1   0   1   0   0    1
1   0   1   0   0   1   0    0
2   0   0   1   0   0   1    0
3   0   0   1   1   0   0    0
4   0   1   0   0   1   0    0


In [92]:
X = data_1.iloc[:, :-1]  # All columns except the last one (Win)
y = data_1.iloc[:, -1]  # The target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [93]:
print(y_test)

19    1
16    1
15    1
26    1
4     0
12    1
37    0
27    1
Name: Win, dtype: int64


In [94]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [95]:
y_pred = model.predict(X_test)
print(X_test)
print(y_pred)
print(y_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

    HP  HR  HS  CP  CR  CS
19   0   0   1   0   1   0
16   0   0   1   0   1   0
15   0   0   1   0   1   0
26   0   0   1   0   1   0
4    0   1   0   0   1   0
12   0   1   0   1   0   0
37   0   0   1   1   0   0
27   0   0   1   0   1   0
[1 1 1 1 0 1 0 1]
19    1
16    1
15    1
26    1
4     0
12    1
37    0
27    1
Name: Win, dtype: int64
Accuracy: 1.00
